# Project 3, Calculating number of customers around BART stations


University of California, Berkeley

Master of Information and Data Science (MIDS) program

W205, Section 01 Annie Cui, Emily Zhou, Shuo Wang

# Overview

Our goal is to increase AGM brand awareness transbay. One of the ways we'll do that is by piloting pick-up points or delivery points at 3 BART stations. In this notebook, we will calcuate the number of customers around each stations.

# BART Map

![Bart Map](bart_map.png)

# Included Modules and Packages

Code cell containing your includes for modules and packages

In [18]:
import math
import numpy as np
import pandas as pd

import psycopg2

from geographiclib.geodesic import Geodesic

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  

In [19]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

Create function to get zip codes around a station within miles and total customer number

In [35]:
def my_station_get_zips(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the customer_number"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, customer_number from zip_count_customer "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    print("\n-------------------------------------------------------------------------------")
    print("  Zip Codes within " + str(miles) + " mile(s) of " + station + " BART Station")
    print("-------------------------------------------------------------------------------\n")
    
    total_customer_number = 0
    
    for row in rows:
        zip = row[0]
        customer_number = row[1]
        print("     zip:", zip, "  customer_number: ", f'{customer_number:10,}')
        total_customer_number += customer_number
        
    
    print("\n-------------------------------------------------------------------------------")
    print("  Total customer_number: ", f'{total_customer_number:10,}')
    print("-------------------------------------------------------------------------------")

In [21]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [22]:
cursor = connection.cursor()

In [23]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

# 1. Table customers

This is the customers table. To give a sense of the data, the first 10 rows are shown.

In [24]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from customers
limit 10

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,customer_id,first_name,last_name,street,city,state,zip,closest_store_id,distance
0,1,Robb,Weaving,5 Ramsey Place,Oakland,CA,94609,1,1
1,2,Robby,Belliard,6 Londonderry Plaza,Oakland,CA,94609,1,1
2,3,Sadella,Caudrelier,548 Mcguire Parkway,Oakland,CA,94609,1,1
3,4,Holmes,Shimmings,99 Kennedy Court,Oakland,CA,94609,1,1
4,5,Beverley,Gubbin,51 Mcbride Drive,Oakland,CA,94609,1,1
5,6,Pavia,Millery,463 Columbus Pass,Oakland,CA,94609,1,1
6,7,Engracia,Jeanon,24 Hanover Court,Oakland,CA,94609,1,1
7,8,Melinda,Stodd,568 Nova Way,Oakland,CA,94609,1,1
8,9,Dinnie,Leek,38 Golf Alley,Oakland,CA,94609,1,1
9,10,Caitrin,Filipowicz,61 Michigan Drive,Oakland,CA,94609,1,1


# 2. count customer number group by zip code
Based on customer table, the number of the customers are grouped by zip code. To give a sense of the data, the first 10 rows are shown.

In [25]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select zip, count(*)
from customers 
group by zip
limit 10

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,count
0,37062,8
1,94066,9
2,75023,7
3,33131,115
4,76180,1
5,76017,4
6,94801,58
7,75025,1
8,94964,34
9,33182,45


# 3. join customer table with zip_codes table

We add latitude and longitude for each zip code by joining customer table with zip_codes table. To give a sense of the data, the first 10 rows are shown.

In [26]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select a.zip, count(a.customer_id), b.latitude, b.longitude
from customers a
     join zip_codes b
       on a.zip = b.zip
group by 1,3,4

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,count,latitude,longitude
0,94065,6,37.5354,-122.2467
1,94590,1,38.1030,-122.2486
2,98105,177,47.6607,-122.2840
3,75042,76,32.9139,-96.6749
4,75203,173,32.7463,-96.8030
...,...,...,...,...
545,94588,1,37.7375,-121.8818
546,75032,7,32.8550,-96.4276
547,33055,47,25.9482,-80.2780
548,94801,58,37.9496,-122.3811


# 4. Drop zip_count_customer table if it exists

In [32]:
connection.rollback()

query = """

drop table if exists zip_count_customer;

"""

cursor.execute(query)

connection.commit()

# 5. Create zip_count_customer table

Create zip_count_customer table by joining customer table with zip_codes table

In [33]:
connection.rollback()

query = """

create table zip_count_customer 

as select a.zip, count(a.customer_id) as customer_number, b.latitude, b.longitude
from customers a
     join zip_codes b
       on a.zip = b.zip
group by 1,3,4

"""

cursor.execute(query)

connection.commit()

# 6. Verify the zip_count_customer table loaded correctly
To give a sense of the data, the first 10 rows are shown.

In [34]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from zip_count_customer
limit 10
"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,zip,customer_number,latitude,longitude
0,94065,6,37.5354,-122.2467
1,94590,1,38.1030,-122.2486
2,98105,177,47.6607,-122.2840
3,75042,76,32.9139,-96.6749
4,75203,173,32.7463,-96.8030
5,94591,5,38.1234,-122.1954
6,37135,8,35.9363,-86.6660
7,94109,75,37.7952,-122.4222
8,75007,45,33.0046,-96.8971
9,33172,82,25.7863,-80.3605


# 7. Load station table into dateframe
To give a sense of the data, the first 10 rows are shown.

In [36]:
df = pd.read_csv ('stations.csv')
df[:10]

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282
1,16th Street Mission,37.764847,-122.420042,287
2,19th Street,37.807869,-122.268980,67
3,24th Street Mission,37.752000,-122.418700,277
4,Antioch,37.996281,-121.783404,0
5,Ashby,37.853068,-122.269957,299
6,Balboa Park,37.721667,-122.447500,48
7,Bay Fair,37.697000,-122.126500,63
8,Berryessa,37.368361,-121.874655,288
9,Castro Valley,37.690748,-122.075679,0


# 8. Get zip codes around all stations within 1 mile and total customer number

The code will give us the following information within 1 mile around all the stations.
-zip code and corresponding number of customers within this zip code
-total number of customers

To give a sense of the data, only the information for first 5 stations are shown.

In [48]:
df1 = df['station']
df1
for station_name in df1[0:5]:
#     print(station_name)
    my_station_get_zips(station_name, 1)


-------------------------------------------------------------------------------
  Zip Codes within 1 mile(s) of 12th Street BART Station
-------------------------------------------------------------------------------

     zip: 94612   customer_number:         161

-------------------------------------------------------------------------------
  Total customer_number:         161
-------------------------------------------------------------------------------

-------------------------------------------------------------------------------
  Zip Codes within 1 mile(s) of 16th Street Mission BART Station
-------------------------------------------------------------------------------

     zip: 94103   customer_number:          77
     zip: 94114   customer_number:          58

-------------------------------------------------------------------------------
  Total customer_number:         135
-------------------------------------------------------------------------------

----------------